In [ ]:
#Seperated Scripts

In [ ]:
# BAB Sheet Script V2.4 = FINAL!!!

In [1]:
# Version 2.4 FINAL

import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.types import VARCHAR, DATE, DECIMAL
import logging
from dateutil.parser import parse
import re

# Logging setup
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler(), logging.FileHandler('load_factor_data.log')]
)
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

# Database connection
engine = create_engine(
    "mssql+pyodbc://JULIANS_LAPTOP\\SQLEXPRESS/CWA_Fund_Database"
    "?driver=ODBC+Driver+18+for+SQL+Server&trusted_connection=yes&TrustServerCertificate=yes"
)

# Configuration for all sheets (unchanged)
config = {
    "path": r"C:\Users\JulianHeron\OneDrive - Credentialed Wealth Advisors\Documents\AQR_Factor_Checks\Betting_Against_Beta_Equity_Factors_Monthly.xlsx",
    "sheets": {
        "BAB": {
            "sheet": "BAB Factors",
            "start_row": 19,
            "paper": "Betting Against Beta (Frazzini and Pedersen, 2014)",
            "factor": "BAB",
            "asset_class": "Equity",
            "headers": ["DATE", "AUS", "AUT", "BEL", "CAN", "CHE", "DEU", "DNK", "ESP", "FIN", "FRA", "GBR", 
                        "GRC", "HKG", "IRL", "ISR", "ITA", "JPN", "NLD", "NOR", "NZL", "PRT", "SGP", "SWE", 
                        "USA", "Global", "Global Ex USA", "Europe", "North America", "Pacific"],
            "columns": ["DATE", "USA", "Global", "Global Ex USA"]
        },
        "MKT": {
            "sheet": "MKT",
            "start_row": 19,
            "paper": "Fama-French Factors",
            "factor": "MKT",
            "asset_class": "Equity",
            "headers": ["DATE", "AUS", "AUT", "BEL", "CAN", "CHE", "DEU", "DNK", "ESP", "FIN", "FRA", "GBR", 
                        "GRC", "HKG", "IRL", "ISR", "ITA", "JPN", "NLD", "NOR", "NZL", "PRT", "SGP", "SWE", 
                        "USA", "Global", "Global Ex USA", "Europe", "North America", "Pacific"],
            "columns": ["DATE", "USA", "Global", "Global Ex USA"]
        },
        "SMB": {
            "sheet": "SMB",
            "start_row": 19,
            "paper": "Fama-French Factors",
            "factor": "SMB",
            "asset_class": "Equity",
            "headers": ["DATE", "AUS", "AUT", "BEL", "CAN", "CHE", "DEU", "DNK", "ESP", "FIN", "FRA", "GBR", 
                        "GRC", "HKG", "IRL", "ISR", "ITA", "JPN", "NLD", "NOR", "NZL", "PRT", "SGP", "SWE", 
                        "USA", "Global", "Global Ex USA", "Europe", "North America", "Pacific"],
            "columns": ["DATE", "USA", "Global", "Global Ex USA"]
        },
        "HML_FF": {
            "sheet": "HML FF",
            "start_row": 19,
            "paper": "Fama-French Factors",
            "factor": "HML_FF",
            "asset_class": "Equity",
            "headers": ["DATE", "AUS", "AUT", "BEL", "CAN", "CHE", "DEU", "DNK", "ESP", "FIN", "FRA", "GBR", 
                        "GRC", "HKG", "IRL", "ISR", "ITA", "JPN", "NLD", "NOR", "NZL", "PRT", "SGP", "SWE", 
                        "USA", "Global", "Global Ex USA", "Europe", "North America", "Pacific"],
            "columns": ["DATE", "USA", "Global", "Global Ex USA"]
        },
        "HML_Devil": {
            "sheet": "HML Devil",
            "start_row": 19,
            "paper": "The Devil's in HML's Details",
            "factor": "HML_Devil",
            "asset_class": "Equity",
            "headers": ["DATE", "AUS", "AUT", "BEL", "CAN", "CHE", "DEU", "DNK", "ESP", "FIN", "FRA", "GBR", 
                        "GRC", "HKG", "IRL", "ISR", "ITA", "JPN", "NLD", "NOR", "NZL", "PRT", "SGP", "SWE", 
                        "USA", "Global", "Global Ex USA", "Europe", "North America", "Pacific"],
            "columns": ["DATE", "USA", "Global", "Global Ex USA"]
        },
        "UMD": {
            "sheet": "UMD",
            "start_row": 19,
            "paper": "On Persistence in Mutual Fund Performance",
            "factor": "UMD",
            "asset_class": "Equity",
            "headers": ["DATE", "AUS", "AUT", "BEL", "CAN", "CHE", "DEU", "DNK", "ESP", "FIN", "FRA", "GBR", 
                        "GRC", "HKG", "IRL", "ISR", "ITA", "JPN", "NLD", "NOR", "NZL", "PRT", "SGP", "SWE", 
                        "USA", "Global", "Global Ex USA", "Europe", "North America", "Pacific"],
            "columns": ["DATE", "USA", "Global", "Global Ex USA"]
        },
        "ME": {
            "sheet": "ME(t-1)",
            "start_row": 19,
            "paper": "AQR Factors",
            "factor": "ME",
            "asset_class": "Equity",
            "headers": ["DATE", "AUT", "HKG", "ESP", "GBR", "ITA", "DEU", "DNK", "NZL", "NLD", "USA", "PRT", 
                        "BEL", "ISR", "GRC", "NOR", "SGP", "CHE", "IRL", "CAN", "FIN", "JPN", "SWE", "FRA", 
                        "AUS", "Global Ex USA", "Global", "Europe", "North America", "Pacific"],
            "columns": ["DATE", "USA", "Global", "Global Ex USA"]
        },
        "RF": {
            "sheet": "RF",
            "start_row": 19,
            "paper": None,
            "factor": "RF",
            "asset_class": "Fixed Income",
            "headers": ["DATE", "Risk Free Rate"],
            "columns": ["DATE", "Risk Free Rate"]
        }
    }
}

def clean_date(date_val):
    """Attempt to clean and parse a date value, returning None if unfixable."""
    if pd.isna(date_val) or str(date_val).strip().lower() in ['', 'nan', 'risk free rate']:
        return None
    try:
        return pd.to_datetime(date_val, errors='coerce')
    except:
        pass
    try:
        return parse(str(date_val), fuzzy=False)
    except:
        pass
    date_str = str(date_val).strip()
    if re.match(r'\d{4}-\d{2}-\d{2}', date_str):
        try:
            year, month, day = map(int, date_str.split('-'))
            if month > 12:
                month = 12
            if day > 31:
                day = 31
            return pd.to_datetime(f"{year}-{month:02d}-{day:02d}")
        except:
            pass
    logger.warning(f"Unable to parse date: {date_val}")
    return None

def process_sheet(sheet_config, parent_path):
    file_path = parent_path
    sheet_name = sheet_config['sheet']
    logger.info(f"Processing {file_path} (sheet {sheet_name})")
    
    try:
        # Read the Excel sheet
        df = pd.read_excel(
            file_path,
            sheet_name=sheet_name,
            header=None,
            skiprows=sheet_config['start_row']
        )
        df.columns = sheet_config['headers']
        logger.info(f"Read {len(df)} rows with columns: {list(df.columns)}")

        # Select only the required columns
        valid_cols = [col for col in sheet_config['columns'] if col in df.columns]
        if not valid_cols:
            logger.error(f"No valid columns found in {file_path} (sheet {sheet_name}). Skipping.")
            return
        df = df[valid_cols]
        logger.info(f"Selected columns: {list(df.columns)}")

        # Rename DATE to date
        df = df.rename(columns={"DATE": "date"})
        
        # Clean and parse dates
        invalid_dates = []
        df['date'] = df['date'].apply(clean_date)
        invalid_indices = df['date'].isna()
        if invalid_indices.any():
            invalid_dates = df[invalid_indices].index.tolist()
            logger.warning(f"Found {len(invalid_dates)} unfixable dates in {sheet_name} at rows: {invalid_dates}")
            logger.debug(f"Sample invalid date rows: {df[invalid_indices].head().to_dict('records')}")
        df = df.dropna(subset=['date'])
        logger.debug(f"After date cleaning, {len(df)} rows remain")

        # Normalize dates to date only
        df['date'] = df['date'].dt.date

        # Melt the DataFrame
        value_vars = [col for col in df.columns if col != 'date']
        region_map = {
            "USA": "USA",
            "Global": "Global",
            "Global Ex USA": "Intl",
            "Risk Free Rate": "USA"
        }
        df_long = df.melt(
            id_vars=['date'],
            value_vars=value_vars,
            var_name='region_col',
            value_name='value'
        )
        
        # Assign factor, region, and other columns
        df_long['factor'] = sheet_config['factor']
        df_long['region'] = df_long['region_col'].map(region_map)
        df_long['asset_class'] = sheet_config['asset_class']
        df_long['associated_paper'] = sheet_config['paper'] if sheet_config['paper'] is not None else 'Unknown'
        
        # Drop unnecessary column and rows with NA values
        df_long = df_long.drop(columns=['region_col']).dropna(subset=['value', 'region'])
        logger.debug(f"Rows after melting and dropping NA: {len(df_long)}")
        
        # Convert date to datetime for SQL compatibility
        df_long['date'] = pd.to_datetime(df_long['date'])
        
        # Reorder columns to match table schema
        df_long = df_long[['factor', 'date', 'associated_paper', 'value', 'region', 'asset_class']]
        
        # Create key for duplicate checking
        df_long['key'] = df_long.apply(
            lambda row: (
                row['factor'],
                row['date'].date(),
                row['associated_paper'],
                row['region']
            ),
            axis=1
        )
        
        # Check for duplicates within the new data
        duplicates = df_long.duplicated(subset=['key'], keep=False)
        if duplicates.any():
            logger.warning(f"Found {duplicates.sum()} duplicate keys within {sheet_name} data: "
                          f"{df_long[duplicates][['factor', 'date', 'region', 'value']].to_dict('records')}")
            df_long = df_long.drop_duplicates(subset=['key'], keep='first')
            logger.debug(f"After removing intra-sheet duplicates, {len(df_long)} rows remain")

        # Insert all data with transaction
        with engine.connect() as connection:
            with connection.begin() as transaction:
                existing = pd.read_sql(
                    "SELECT factor, date, associated_paper, region FROM factor_returns",
                    connection
                )
                logger.debug(f"Raw existing data sample (first 5 rows): {existing.head().to_dict('records')}")
                existing['date'] = pd.to_datetime(existing['date'], errors='coerce').dt.date
                existing_keys = set(
                    (row['factor'], row['date'], row['associated_paper'], row['region'])
                    for _, row in existing.iterrows()
                    if pd.notna(row['date']) and pd.notna(row['region'])
                )
                logger.debug(f"Existing keys sample (first 5): {list(existing_keys)[:5]}")
                logger.debug(f"Total existing keys: {len(existing_keys)}")

                df_new = df_long[~df_long['key'].isin(existing_keys)].drop(columns=['key'])
        
                if not df_new.empty:
                    logger.debug(f"New rows to insert: {len(df_new)}. Sample (first 5): "
                                f"{df_new[['factor', 'date', 'region']].head().to_dict('records')}")
                    
                    try:
                        df_new.to_sql(
                            'factor_returns',
                            connection,
                            if_exists='append',
                            index=False,
                            dtype={
                                'factor': VARCHAR(50),
                                'date': DATE,
                                'associated_paper': VARCHAR(100),
                                'value': DECIMAL(15, 6),
                                'region': VARCHAR(50),
                                'asset_class': VARCHAR(50)
                            }
                        )
                        logger.info(f"Inserted {len(df_new)} new rows into factor_returns from {file_path} (sheet {sheet_name})")
                    except Exception as e:
                        logger.error(f"Error inserting {len(df_new)} rows for {sheet_name}: "
                                    f"{type(e).__name__}: {str(e)}. First row: {df_new.iloc[0].to_dict()}")
                        transaction.rollback()
                        return
                else:
                    logger.info(f"No new rows to load into factor_returns from {file_path} (sheet {sheet_name})")
                    logger.debug(f"Input date range: {df_long['date'].min()} to {df_long['date'].max()}")
                    logger.debug(f"Database date range: {existing['date'].min() if not existing.empty else 'N/A'} to "
                                f"{existing['date'].max() if not existing.empty else 'N/A'}")
        
        if invalid_dates:
            logger.info(f"Review {sheet_name} for unfixable dates at rows: {invalid_dates}")
            
    except Exception as e:
        logger.error(f"Error processing {file_path} (sheet {sheet_name}): {type(e).__name__}: {str(e)}")

# Process all sheets
for sheet_key, sheet_config in config['sheets'].items():
    process_sheet(sheet_config, config['path'])

logger.info("Factor data processing complete!")

2025-04-11 16:04:40,454 - INFO - Processing C:\Users\JulianHeron\OneDrive - Credentialed Wealth Advisors\Documents\AQR_Factor_Checks\Betting_Against_Beta_Equity_Factors_Monthly.xlsx (sheet BAB Factors)
2025-04-11 16:04:41,253 - INFO - Read 1129 rows with columns: ['DATE', 'AUS', 'AUT', 'BEL', 'CAN', 'CHE', 'DEU', 'DNK', 'ESP', 'FIN', 'FRA', 'GBR', 'GRC', 'HKG', 'IRL', 'ISR', 'ITA', 'JPN', 'NLD', 'NOR', 'NZL', 'PRT', 'SGP', 'SWE', 'USA', 'Global', 'Global Ex USA', 'Europe', 'North America', 'Pacific']
2025-04-11 16:04:41,261 - INFO - Selected columns: ['DATE', 'USA', 'Global', 'Global Ex USA']
2025-04-11 16:04:41,976 - DEBUG - After date cleaning, 1129 rows remain
2025-04-11 16:04:41,989 - DEBUG - Rows after melting and dropping NA: 2039
2025-04-11 16:04:42,300 - DEBUG - Raw existing data sample (first 5 rows): [{'factor': 'BAB', 'date': datetime.date(1930, 12, 31), 'associated_paper': 'Betting Against Beta (Frazzini and Pedersen, 2014)', 'region': 'USA'}, {'factor': 'BAB', 'date': date

In [ ]:
# TSMOM Script FINAL

In [2]:
# TSMOM Version 2.0 FINAL

import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.types import VARCHAR, DATE, DECIMAL
import logging
from dateutil.parser import parse
import re

# Logging setup
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler(), logging.FileHandler('load_tsmom_data.log')]
)
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

# Database connection
engine = create_engine(
    "mssql+pyodbc://JULIANS_LAPTOP\\SQLEXPRESS/CWA_Fund_Database"
    "?driver=ODBC+Driver+18+for+SQL+Server&trusted_connection=yes&TrustServerCertificate=yes"
)

# Configuration
config = {
    "path": r"C:\Users\JulianHeron\Downloads\Time Series Momentum Factors Monthly.xlsx",
    "sheet": "TSMOM Factors",
    "start_row": 18,
    "paper": "Time Series Momentum (Moskowitz, Ooi, and Pedersen, 2012)",
    "headers": ["DATE", "TSMOM", "TSMOM^CM", "TSMOM^EQ", "TSMOM^FI", "TSMOM^FX"],
    "columns": {
        "DATE": "DATE",
        "TSMOM": "TSM-MA",
        "TSMOM^CM": "TSM-Com",
        "TSMOM^EQ": "TSM-EQ",
        "TSMOM^FI": "TSM-FI",
        "TSMOM^FX": "TSM-FX"
    }
}

def clean_date(date_val):
    """Attempt to clean and parse a date value, returning None if unfixable."""
    if pd.isna(date_val) or str(date_val).strip().lower() in ['', 'nan']:
        return None
    try:
        return pd.to_datetime(date_val, errors='coerce')
    except:
        pass
    try:
        return parse(str(date_val), fuzzy=False)
    except:
        pass
    date_str = str(date_val).strip()
    if re.match(r'\d{4}-\d{2}-\d{2}', date_str):
        try:
            year, month, day = map(int, date_str.split('-'))
            if month > 12:
                month = 12
            if day > 31:
                day = 31
            return pd.to_datetime(f"{year}-{month:02d}-{day:02d}")
        except:
            pass
    logger.warning(f"Unable to parse date: {date_val}")
    return None

def process_tsmom(config):
    file_path = config['path']
    sheet_name = config['sheet']
    logger.info(f"Processing {file_path} (sheet {sheet_name})")
    
    try:
        # Read the Excel sheet
        df = pd.read_excel(
            file_path,
            sheet_name=sheet_name,
            header=None,
            skiprows=config['start_row']
        )
        df.columns = config['headers']
        logger.info(f"Read {len(df)} rows with columns: {list(df.columns)}")

        # Select and rename columns
        date_col = 'DATE'
        valid_cols = [col for col in config['columns'].keys() if col in df.columns]
        if not valid_cols or date_col not in valid_cols:
            logger.error(f"No valid columns or missing DATE in {file_path} (sheet {sheet_name}). Skipping.")
            return
        df = df[valid_cols]
        df.columns = ['date' if col == date_col else config['columns'][col] for col in df.columns]
        logger.info(f"Columns after renaming: {list(df.columns)}")

        # Clean and parse dates
        invalid_dates = []
        df['date'] = df['date'].apply(clean_date)
        invalid_indices = df['date'].isna()
        if invalid_indices.any():
            invalid_dates = df[invalid_indices].index.tolist()
            logger.warning(f"Found {len(invalid_dates)} unfixable dates in {sheet_name} at rows: {invalid_dates}")
            logger.debug(f"Sample invalid date rows: {df[invalid_indices].head().to_dict('records')}")
        df = df.dropna(subset=['date'])
        logger.debug(f"After date cleaning, {len(df)} rows remain")

        # Normalize dates to date only
        df['date'] = df['date'].dt.date

        # Melt the DataFrame
        value_vars = [col for col in df.columns if col != 'date']
        df_long = df.melt(
            id_vars=['date'],
            value_vars=value_vars,
            var_name='factor',
            value_name='value'
        )
        
        # Assign region and other columns
        df_long['region'] = "Global"
        df_long['asset_class'] = df_long['factor'].map({
            "TSM-MA": "Multi-Asset",
            "TSM-Com": "Commodities",
            "TSM-EQ": "Equity",
            "TSM-FI": "Fixed Income",
            "TSM-FX": "Currencies"
        })
        df_long['associated_paper'] = config['paper']
        df_long = df_long.dropna(subset=['value'])
        logger.debug(f"Rows after melting and dropping NA: {len(df_long)}")
        
        # Convert date to datetime for SQL compatibility
        df_long['date'] = pd.to_datetime(df_long['date'])
        
        # Reorder columns to match table schema
        df_long = df_long[['factor', 'date', 'associated_paper', 'value', 'region', 'asset_class']]
        
        # Create key for duplicate checking
        df_long['key'] = df_long.apply(
            lambda row: (
                row['factor'],
                row['date'].date(),
                row['associated_paper'],
                row['region']
            ),
            axis=1
        )
        
        # Check for duplicates within the new data
        duplicates = df_long.duplicated(subset=['key'], keep=False)
        if duplicates.any():
            logger.warning(f"Found {duplicates.sum()} duplicate keys within {sheet_name} data: "
                          f"{df_long[duplicates][['factor', 'date', 'region', 'value']].to_dict('records')}")
            df_long = df_long.drop_duplicates(subset=['key'], keep='first')
            logger.debug(f"After removing intra-sheet duplicates, {len(df_long)} rows remain")

        # Insert all data with transaction
        with engine.connect() as connection:
            with connection.begin() as transaction:
                existing = pd.read_sql(
                    "SELECT factor, date, associated_paper, region FROM factor_returns",
                    connection
                )
                logger.debug(f"Raw existing data sample (first 5 rows): {existing.head().to_dict('records')}")
                existing['date'] = pd.to_datetime(existing['date'], errors='coerce').dt.date
                existing_keys = set(
                    (row['factor'], row['date'], row['associated_paper'], row['region'])
                    for _, row in existing.iterrows()
                    if pd.notna(row['date']) and pd.notna(row['region'])
                )
                logger.debug(f"Existing keys sample (first 5): {list(existing_keys)[:5]}")
                logger.debug(f"Total existing keys: {len(existing_keys)}")

                df_new = df_long[~df_long['key'].isin(existing_keys)].drop(columns=['key'])
        
                if not df_new.empty:
                    logger.debug(f"New rows to insert: {len(df_new)}. Sample (first 5): "
                                f"{df_new[['factor', 'date', 'region']].head().to_dict('records')}")
                    
                    try:
                        df_new.to_sql(
                            'factor_returns',
                            connection,
                            if_exists='append',
                            index=False,
                            dtype={
                                'factor': VARCHAR(50),
                                'date': DATE,
                                'associated_paper': VARCHAR(100),
                                'value': DECIMAL(15, 6),
                                'region': VARCHAR(50),
                                'asset_class': VARCHAR(50)
                            }
                        )
                        logger.info(f"Inserted {len(df_new)} new rows into factor_returns from {file_path} (sheet {sheet_name})")
                    except Exception as e:
                        logger.error(f"Error inserting {len(df_new)} rows for {sheet_name}: "
                                    f"{type(e).__name__}: {str(e)}. First row: {df_new.iloc[0].to_dict()}")
                        transaction.rollback()
                        return
                else:
                    logger.info(f"No new rows to load into factor_returns from {file_path} (sheet {sheet_name})")
                    logger.debug(f"Input date range: {df_long['date'].min()} to {df_long['date'].max()}")
                    logger.debug(f"Database date range: {existing['date'].min() if not existing.empty else 'N/A'} to "
                                f"{existing['date'].max() if not existing.empty else 'N/A'}")
        
        if invalid_dates:
            logger.info(f"Review {sheet_name} for unfixable dates at rows: {invalid_dates}")
            
    except Exception as e:
        logger.error(f"Error processing {file_path} (sheet {sheet_name}): {type(e).__name__}: {str(e)}")

# Run the script
process_tsmom(config)
logger.info("TSMOM data processing complete!")

2025-04-11 16:24:06,533 - INFO - Processing C:\Users\JulianHeron\Downloads\Time Series Momentum Factors Monthly.xlsx (sheet TSMOM Factors)
2025-04-11 16:24:06,811 - INFO - Read 481 rows with columns: ['DATE', 'TSMOM', 'TSMOM^CM', 'TSMOM^EQ', 'TSMOM^FI', 'TSMOM^FX']
2025-04-11 16:24:06,814 - INFO - Columns after renaming: ['date', 'TSM-MA', 'TSM-Com', 'TSM-EQ', 'TSM-FI', 'TSM-FX']
2025-04-11 16:24:06,825 - DEBUG - After date cleaning, 481 rows remain
2025-04-11 16:24:06,837 - DEBUG - Rows after melting and dropping NA: 2405
2025-04-11 16:24:07,151 - DEBUG - Raw existing data sample (first 5 rows): [{'factor': 'BAB', 'date': datetime.date(1930, 12, 31), 'associated_paper': 'Betting Against Beta (Frazzini and Pedersen, 2014)', 'region': 'USA'}, {'factor': 'BAB', 'date': datetime.date(1931, 1, 31), 'associated_paper': 'Betting Against Beta (Frazzini and Pedersen, 2014)', 'region': 'USA'}, {'factor': 'BAB', 'date': datetime.date(1931, 2, 28), 'associated_paper': 'Betting Against Beta (Frazz

In [ ]:
#QMJ script FINAL

In [6]:
# QMJ Version 2.1 FINAL

import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.types import VARCHAR, DATE, DECIMAL
import logging
from dateutil.parser import parse
import re

# Logging setup
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler(), logging.FileHandler('load_qmj_data.log')]
)
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

# Database connection
engine = create_engine(
    "mssql+pyodbc://JULIANS_LAPTOP\\SQLEXPRESS/CWA_Fund_Database"
    "?driver=ODBC+Driver+18+for+SQL+Server&trusted_connection=yes&TrustServerCertificate=yes"
)

# Configuration
config = {
    "path": r"C:\Users\JulianHeron\Downloads\Quality Minus Junk 10 QualitySorted Portfolios Monthly.xlsx",
    "sheet": "10 Portfolios Formed on Quality",
    "start_row": 19,
    "paper": "Quality Minus Junk (Asness, Frazzini and Pedersen, 2014)",
    "headers": ["DATE", "P1 (low quality)", "P2", "P3", "P4", "P5", "P6", "P7", "P8", "P9", "P10 (high quality)", "P10-P1", 
                "P1 (low quality).1", "P2.1", "P3.1", "P4.1", "P5.1", "P6.1", "P7.1", "P8.1", "P9.1", "P10 (high quality).1", "P10-P1.1"],
    "columns": {
        "DATE": "DATE",
        "P1 (low quality)": "QMJ_P1",
        "P2": "QMJ_P2",
        "P3": "QMJ_P3",
        "P4": "QMJ_P4",
        "P5": "QMJ_P5",
        "P6": "QMJ_P6",
        "P7": "QMJ_P7",
        "P8": "QMJ_P8",
        "P9": "QMJ_P9",
        "P10 (high quality)": "QMJ_P10",
        "P10-P1": "QMJ",
        "P1 (low quality).1": "QMJ_P1",
        "P2.1": "QMJ_P2",
        "P3.1": "QMJ_P3",
        "P4.1": "QMJ_P4",
        "P5.1": "QMJ_P5",
        "P6.1": "QMJ_P6",
        "P7.1": "QMJ_P7",
        "P8.1": "QMJ_P8",
        "P9.1": "QMJ_P9",
        "P10 (high quality).1": "QMJ_P10",
        "P10-P1.1": "QMJ"
    },
    "region_map": {
        "P1 (low quality)": "USA",
        "P2": "USA",
        "P3": "USA",
        "P4": "USA",
        "P5": "USA",
        "P6": "USA",
        "P7": "USA",
        "P8": "USA",
        "P9": "USA",
        "P10 (high quality)": "USA",
        "P10-P1": "USA",
        "P1 (low quality).1": "Global",
        "P2.1": "Global",
        "P3.1": "Global",
        "P4.1": "Global",
        "P5.1": "Global",
        "P6.1": "Global",
        "P7.1": "Global",
        "P8.1": "Global",
        "P9.1": "Global",
        "P10 (high quality).1": "Global",
        "P10-P1.1": "Global"
    }
}

def clean_date(date_val):
    """Attempt to clean and parse a date value, returning None if unfixable."""
    if pd.isna(date_val) or str(date_val).strip().lower() in ['', 'nan']:
        return None
    try:
        return pd.to_datetime(date_val, errors='coerce')
    except:
        pass
    try:
        return parse(str(date_val), fuzzy=False)
    except:
        pass
    date_str = str(date_val).strip()
    if re.match(r'\d{4}-\d{2}-\d{2}', date_str):
        try:
            year, month, day = map(int, date_str.split('-'))
            if month > 12:
                month = 12
            if day > 31:
                day = 31
            return pd.to_datetime(f"{year}-{month:02d}-{day:02d}")
        except:
            pass
    logger.warning(f"Unable to parse date: {date_val}")
    return None

def process_qmj(config):
    file_path = config['path']
    sheet_name = config['sheet']
    logger.info(f"Processing {file_path} (sheet {sheet_name})")
    
    try:
        # Read the Excel sheet
        df = pd.read_excel(
            file_path,
            sheet_name=sheet_name,
            header=None,
            skiprows=config['start_row']
        )
        df.columns = config['headers']
        logger.info(f"Read {len(df)} rows with columns: {list(df.columns)}")

        # Set Global columns to NaN before row 404 (index 384 = 404 - 19 - 1)
        global_start_row = 404 - config['start_row'] - 1
        global_cols = [col for col in df.columns if col.endswith('.1') or col == 'P10-P1.1']
        if global_cols:
            non_na_count = df.loc[:global_start_row-1, global_cols].notna().sum().sum()
            if non_na_count > 0:
                logger.warning(f"Found {non_na_count} non-NaN values in Global columns before row 404")
            df.loc[:global_start_row-1, global_cols] = pd.NA
            logger.info(f"Set rows 0–{global_start_row-1} for Global columns {global_cols} to NaN")

        # Split into US and Global DataFrames
        date_col = 'DATE'
        us_cols = [col for col in df.columns if col == date_col or (col in config['columns'] and not col.endswith('.1') and col != 'P10-P1.1')]
        global_cols = [date_col] + [col for col in df.columns if col.endswith('.1') or col == 'P10-P1.1']
        
        df_us = df[us_cols].copy()
        df_global = df[global_cols].copy()
        logger.info(f"US columns: {list(df_us.columns)}")
        logger.info(f"Global columns: {list(df_global.columns)}")

        # Rename columns
        df_us.columns = ['date' if col == date_col else config['columns'][col] for col in df_us.columns]
        df_global.columns = ['date' if col == date_col else config['columns'][col] for col in df_global.columns]
        logger.info(f"US columns after renaming: {list(df_us.columns)}")
        logger.info(f"Global columns after renaming: {list(df_global.columns)}")

        # Clean and parse dates
        invalid_dates = []
        for df_temp, label in [(df_us, 'US'), (df_global, 'Global')]:
            df_temp['date'] = df_temp['date'].apply(clean_date)
            invalid_indices = df_temp['date'].isna()
            if invalid_indices.any():
                invalid_dates.extend(df_temp[invalid_indices].index.tolist())
                logger.warning(f"Found {len(df_temp[invalid_indices])} unfixable dates in {sheet_name} ({label}) at rows: {df_temp[invalid_indices].index.tolist()}")
                logger.debug(f"Sample invalid date rows ({label}): {df_temp[invalid_indices].head().to_dict('records')}")
            df_temp.dropna(subset=['date'], inplace=True)
            logger.debug(f"After date cleaning, {len(df_temp)} {label} rows remain")

        # Normalize dates to date only
        df_us['date'] = df_us['date'].dt.date
        df_global['date'] = df_global['date'].dt.date

        # Melt DataFrames separately
        us_value_vars = [col for col in df_us.columns if col != 'date']
        global_value_vars = [col for col in df_global.columns if col != 'date']
        
        df_us_long = df_us.melt(
            id_vars=['date'],
            value_vars=us_value_vars,
            var_name='factor',
            value_name='value'
        )
        df_us_long['region'] = 'USA'
        
        df_global_long = df_global.melt(
            id_vars=['date'],
            value_vars=global_value_vars,
            var_name='factor',
            value_name='value'
        )
        df_global_long['region'] = 'Global'
        
        # Concatenate and clean
        df_long = pd.concat([df_us_long, df_global_long], ignore_index=True)
        df_long['asset_class'] = "Equity"
        df_long['associated_paper'] = config['paper']
        df_long = df_long.dropna(subset=['value'])
        logger.info(f"Rows after melting and dropping NA: {len(df_long)}")
        logger.info(f"USA rows: {len(df_long[df_long['region'] == 'USA'])}")
        logger.info(f"Global rows: {len(df_long[df_long['region'] == 'Global'])}")
        
        # Convert date to datetime for SQL compatibility
        df_long['date'] = pd.to_datetime(df_long['date'])
        
        # Reorder columns to match table schema
        df_long = df_long[['factor', 'date', 'associated_paper', 'value', 'region', 'asset_class']]
        
        # Create key for duplicate checking
        df_long['key'] = df_long.apply(
            lambda row: (
                row['factor'],
                row['date'].date(),
                row['associated_paper'],
                row['region']
            ),
            axis=1
        )
        
        # Check for duplicates within the new data
        duplicates = df_long.duplicated(subset=['key'], keep=False)
        if duplicates.any():
            logger.warning(f"Found {duplicates.sum()} duplicate keys within {sheet_name} data. Sample (first 5): "
                          f"{df_long[duplicates][['factor', 'date', 'region', 'value']].head(5).to_dict('records')}")
            df_long = df_long.drop_duplicates(subset=['key'], keep='first')
            logger.debug(f"After removing intra-sheet duplicates, {len(df_long)} rows remain")

        # Insert all data with transaction
        with engine.connect() as connection:
            with connection.begin() as transaction:
                existing = pd.read_sql(
                    "SELECT factor, date, associated_paper, region FROM factor_returns",
                    connection
                )
                logger.debug(f"Raw existing data sample (first 5 rows): {existing.head().to_dict('records')}")
                existing['date'] = pd.to_datetime(existing['date'], errors='coerce').dt.date
                existing_keys = set(
                    (row['factor'], row['date'], row['associated_paper'], row['region'])
                    for _, row in existing.iterrows()
                    if pd.notna(row['date']) and pd.notna(row['region'])
                )
                logger.debug(f"Existing keys sample (first 5): {list(existing_keys)[:5]}")
                logger.debug(f"Total existing keys: {len(existing_keys)}")

                df_new = df_long[~df_long['key'].isin(existing_keys)].drop(columns=['key'])
        
                if not df_new.empty:
                    logger.debug(f"New rows to insert: {len(df_new)}. Sample (first 5): "
                                f"{df_new[['factor', 'date', 'region']].head().to_dict('records')}")
                    
                    try:
                        df_new.to_sql(
                            'factor_returns',
                            connection,
                            if_exists='append',
                            index=False,
                            dtype={
                                'factor': VARCHAR(50),
                                'date': DATE,
                                'associated_paper': VARCHAR(100),
                                'value': DECIMAL(15, 6),
                                'region': VARCHAR(50),
                                'asset_class': VARCHAR(50)
                            }
                        )
                        logger.info(f"Inserted {len(df_new)} new rows into factor_returns from {file_path} (sheet {sheet_name})")
                    except Exception as e:
                        logger.error(f"Error inserting {len(df_new)} rows for {sheet_name}: "
                                    f"{type(e).__name__}: {str(e)}. First row: {df_new.iloc[0].to_dict()}")
                        transaction.rollback()
                        return
                else:
                    logger.info(f"No new rows to load into factor_returns from {file_path} (sheet {sheet_name})")
                    logger.debug(f"Input date range: {df_long['date'].min()} to {df_long['date'].max()}")
                    logger.debug(f"Database date range: {existing['date'].min() if not existing.empty else 'N/A'} to "
                                f"{existing['date'].max() if not existing.empty else 'N/A'}")
        
        if invalid_dates:
            logger.info(f"Review {sheet_name} for unfixable dates at rows: {invalid_dates}")
            
    except Exception as e:
        logger.error(f"Error processing {file_path} (sheet {sheet_name}): {type(e).__name__}: {str(e)}")

# Run the script
process_qmj(config)
logger.info("QMJ data processing complete!")

2025-04-13 18:53:00,684 - INFO - Processing C:\Users\JulianHeron\Downloads\Quality Minus Junk 10 QualitySorted Portfolios Monthly.xlsx (sheet 10 Portfolios Formed on Quality)
2025-04-13 18:53:00,967 - INFO - Read 809 rows with columns: ['DATE', 'P1 (low quality)', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10 (high quality)', 'P10-P1', 'P1 (low quality).1', 'P2.1', 'P3.1', 'P4.1', 'P5.1', 'P6.1', 'P7.1', 'P8.1', 'P9.1', 'P10 (high quality).1', 'P10-P1.1']
2025-04-13 18:53:00,970 - INFO - Set rows 0–383 for Global columns ['P1 (low quality).1', 'P2.1', 'P3.1', 'P4.1', 'P5.1', 'P6.1', 'P7.1', 'P8.1', 'P9.1', 'P10 (high quality).1', 'P10-P1.1'] to NaN
2025-04-13 18:53:00,972 - INFO - US columns: ['DATE', 'P1 (low quality)', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10 (high quality)', 'P10-P1']
2025-04-13 18:53:00,973 - INFO - Global columns: ['DATE', 'P1 (low quality).1', 'P2.1', 'P3.1', 'P4.1', 'P5.1', 'P6.1', 'P7.1', 'P8.1', 'P9.1', 'P10 (high quality).1', 'P10-P1.1']
20

In [ ]:
# Century Factors Script FINAL

In [1]:
# Century Version 2.4 FINAL

import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.types import VARCHAR, DATE, DECIMAL
import logging
from dateutil.parser import parse
import re

# Logging setup
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler(), logging.FileHandler('load_century_data.log')]
)
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

# Database connection
engine = create_engine(
    "mssql+pyodbc://JULIANS_LAPTOP\\SQLEXPRESS/CWA_Fund_Database"
    "?driver=ODBC+Driver+18+for+SQL+Server&trusted_connection=yes&TrustServerCertificate=yes"
)

# Configuration
config = {
    "path": r"C:\Users\JulianHeron\Downloads\Century of Factor Premia Monthly (1).xlsx",
    "sheet": 0,
    "start_row": 18,
    "paper": "Century of Factor Premia Monthly-Ilmanen et al. (2021)",
    "headers": ["DATE", "US Stock Selection Value", "US Stock Selection Momentum", "US Stock Selection Defensive", "US Stock Selection Multi-style", 
                "Intl Stock Selection Value", "Intl Stock Selection Momentum", "Intl Stock Selection Defensive", "Intl Stock Selection Multi-style", 
                "Equity indices Value", "Equity indices Momentum", "Equity indices Carry", "Equity indices Defensive", "Equity indices Multi-style", 
                "Fixed income Value", "Fixed income Momentum", "Fixed income Carry", "Fixed income Defensive", "Fixed income Multi-style", 
                "Currencies Value", "Currencies Momentum", "Currencies Carry", "Currencies Multi-style", 
                "Commodities Value", "Commodities Momentum", "Commodities Carry", "Commodities Multi-style", 
                "All Stock Selection Value", "All Stock Selection Momentum", "All Stock Selection Defensive", "All Stock Selection Multi-style", 
                "All Macro Value", "All Macro Momentum", "All Macro Carry", "All Macro Defensive", "All Macro Multi-style", 
                "All asset classes Value", "All asset classes Momentum", "All asset classes Carry", "All asset classes Defensive", "All asset classes Multi-style", 
                "Equity indices Market", "Fixed income Market", "Commodities Market", "All Macro Market"]
}

def clean_date(date_val):
    """Attempt to clean and parse a date value, returning None if unfixable."""
    if pd.isna(date_val) or str(date_val).strip().lower() in ['', 'nan']:
        return None
    try:
        return pd.to_datetime(date_val, errors='coerce')
    except:
        pass
    try:
        return parse(str(date_val), fuzzy=False)
    except:
        pass
    date_str = str(date_val).strip()
    if re.match(r'\d{4}-\d{2}-\d{2}', date_str):
        try:
            year, month, day = map(int, date_str.split('-'))
            if month > 12:
                month = 12
            if day > 31:
                day = 31
            return pd.to_datetime(f"{year}-{month:02d}-{day:02d}")
        except:
            pass
    logger.warning(f"Unable to parse date: {date_val}")
    return None

def process_century(config):
    file_path = config['path']
    sheet_name = config['sheet']
    logger.info(f"Processing {file_path} (sheet {sheet_name})")
    
    try:
        # Read the Excel sheet
        df = pd.read_excel(
            file_path,
            sheet_name=sheet_name,
            header=None,
            skiprows=config['start_row']
        )
        df.columns = config['headers']
        logger.info(f"Read {len(df)} rows with columns: {list(df.columns)}")

        # Clean and parse dates
        date_col = 'DATE'
        df['date'] = df[date_col].apply(clean_date)
        invalid_indices = df['date'].isna()
        if invalid_indices.any():
            invalid_dates = df[invalid_indices].index.tolist()
            logger.warning(f"Found {len(invalid_dates)} unfixable dates in {sheet_name} at rows: {invalid_dates}")
            logger.debug(f"Sample invalid date rows: {df[invalid_indices].head().to_dict('records')}")
        df = df.dropna(subset=['date'])
        logger.debug(f"After date cleaning, {len(df)} rows remain")

        # Normalize dates to date only
        df['date'] = df['date'].dt.date

        # Melt the DataFrame, excluding DATE
        value_vars = [col for col in df.columns if col != date_col]
        df_long = df.melt(
            id_vars=['date'],
            value_vars=value_vars,
            var_name='portfolio',
            value_name='value'
        )
        
        # Assign columns for aqr_century_factors
        df_long['factor'] = df_long['portfolio'].apply(
            lambda x: 'Value' if 'Value' in x else 'Momentum' if 'Momentum' in x else 'Defensive' if 'Defensive' in x 
            else 'Carry' if 'Carry' in x else 'Multi-Style' if 'Multi-style' in x else 'Market' if 'Market' in x else 'Unknown'
        )
        df_long['region'] = df_long['portfolio'].apply(
            lambda x: 'USA' if 'US' in x else 'International' if 'Intl' in x else 'Global'
        )
        df_long['asset_class'] = df_long['portfolio'].apply(
            lambda x: 'Equity' if any(s in x for s in ['Stock Selection', 'Equity indices']) 
            else 'Fixed Income' if 'Fixed income' in x 
            else 'Currencies' if 'Currencies' in x 
            else 'Commodities' if 'Commodities' in x 
            else 'Multi-Asset' if any(s in x for s in ['All asset classes', 'All Macro']) 
            else 'Equity' if 'Market' in x 
            else 'Equity'  # Fallback
        )
        df_long['associated_paper'] = config['paper']
        df_long = df_long.dropna(subset=['value'])
        logger.debug(f"Rows after melting and dropping NA: {len(df_long)}")
        
        # Convert date to datetime for SQL compatibility
        df_long['date'] = pd.to_datetime(df_long['date'])
        
        # Reorder columns to match aqr_century_factors schema
        df_long = df_long[['factor', 'portfolio', 'asset_class', 'region', 'date', 'value', 'associated_paper']]
        
        # Create key for duplicate checking
        df_long['key'] = df_long.apply(
            lambda row: (
                row['factor'],
                row['portfolio'],
                row['date'].date(),
                row['associated_paper'],
                row['region'] if pd.notna(row['region']) else ''
            ),
            axis=1
        )
        
        # Check for duplicates within the new data
        duplicates = df_long.duplicated(subset=['key'], keep=False)
        if duplicates.any():
            logger.warning(f"Found {duplicates.sum()} duplicate keys within {sheet_name} data. Sample (first 5): "
                          f"{df_long[duplicates][['factor', 'portfolio', 'date', 'region', 'value']].head(5).to_dict('records')}")
            df_long = df_long.drop_duplicates(subset=['key'], keep='first')
            logger.debug(f"After removing intra-sheet duplicates, {len(df_long)} rows remain")

        # Insert all data with transaction into aqr_century_factors
        with engine.connect() as connection:
            with connection.begin() as transaction:
                existing = pd.read_sql(
                    "SELECT factor, portfolio, date, associated_paper, region FROM aqr_century_factors",
                    connection
                )
                logger.debug(f"Raw existing data sample (first 5 rows): {existing.head().to_dict('records')}")
                existing['date'] = pd.to_datetime(existing['date'], errors='coerce').dt.date
                existing_keys = set(
                    (row['factor'], row['portfolio'], row['date'], row['associated_paper'], row['region'] if pd.notna(row['region']) else '')
                    for _, row in existing.iterrows()
                    if pd.notna(row['date'])
                )
                logger.debug(f"Existing keys sample (first 5): {list(existing_keys)[:5]}")
                logger.debug(f"Total existing keys: {len(existing_keys)}")

                df_new = df_long[~df_long['key'].isin(existing_keys)].drop(columns=['key'])
        
                if not df_new.empty:
                    logger.debug(f"New rows to insert: {len(df_new)}. Sample (first 5): "
                                f"{df_new[['factor', 'portfolio', 'region']].head().to_dict('records')}")
                    
                    try:
                        df_new.to_sql(
                            'aqr_century_factors',
                            connection,
                            if_exists='append',
                            index=False,
                            dtype={
                                'factor': VARCHAR(50),
                                'portfolio': VARCHAR(100),
                                'asset_class': VARCHAR(50),
                                'region': VARCHAR(50),
                                'date': DATE,
                                'value': DECIMAL(15, 6),
                                'associated_paper': VARCHAR(150)
                            }
                        )
                        logger.info(f"Inserted {len(df_new)} new rows into aqr_century_factors from {file_path} (sheet {sheet_name})")
                    except Exception as e:
                        logger.error(f"Error inserting {len(df_new)} rows for {sheet_name}: "
                                    f"{type(e).__name__}: {str(e)}. First row: {df_new.iloc[0].to_dict()}")
                        transaction.rollback()
                        return
                else:
                    logger.info(f"No new rows to load into aqr_century_factors from {file_path} (sheet {sheet_name})")
                    logger.debug(f"Input date range: {df_long['date'].min()} to {df_long['date'].max()}")
                    logger.debug(f"Database date range: {existing['date'].min() if not existing.empty else 'N/A'} to "
                                f"{existing['date'].max() if not existing.empty else 'N/A'}")
        
        if invalid_dates:
            logger.info(f"Review {sheet_name} for unfixable dates at rows: {invalid_dates}")
            
    except Exception as e:
        logger.error(f"Error processing {file_path} (sheet {sheet_name}): {type(e).__name__}: {str(e)}")

# Run the script
process_century(config)
logger.info("Century data processing complete!")

2025-04-13 18:31:26,274 - INFO - Processing C:\Users\JulianHeron\Downloads\Century of Factor Premia Monthly (1).xlsx (sheet 0)
2025-04-13 18:31:26,955 - INFO - Read 1183 rows with columns: ['DATE', 'US Stock Selection Value', 'US Stock Selection Momentum', 'US Stock Selection Defensive', 'US Stock Selection Multi-style', 'Intl Stock Selection Value', 'Intl Stock Selection Momentum', 'Intl Stock Selection Defensive', 'Intl Stock Selection Multi-style', 'Equity indices Value', 'Equity indices Momentum', 'Equity indices Carry', 'Equity indices Defensive', 'Equity indices Multi-style', 'Fixed income Value', 'Fixed income Momentum', 'Fixed income Carry', 'Fixed income Defensive', 'Fixed income Multi-style', 'Currencies Value', 'Currencies Momentum', 'Currencies Carry', 'Currencies Multi-style', 'Commodities Value', 'Commodities Momentum', 'Commodities Carry', 'Commodities Multi-style', 'All Stock Selection Value', 'All Stock Selection Momentum', 'All Stock Selection Defensive', 'All Stock S

In [ ]:
# Commodities for the long run

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.types import VARCHAR, DATE, DECIMAL
import logging

# Logging setup
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler(), logging.FileHandler('load_com_data.log')]
)
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

# Database connection
engine = create_engine(
    "mssql+pyodbc://JULIANS_LAPTOP\\SQLEXPRESS/CWA_Fund_Database"
    "?driver=ODBC+Driver+18+for+SQL+Server&trusted_connection=yes&TrustServerCertificate=yes"
)

# Configuration
config = {
    "path": r"C:\Users\JulianHeron\Downloads\Commodities for the Long Run Index Level Data Monthly.xlsx",
    "sheet": 0,
    "start_row": 11,
    "paper": "Commodities for the Long Run",
    "headers": ["DATE", "Excess return of equal-weight commodities portfolio", "Excess spot return of equal-weight commodities portfolio", "Interest rate adjusted carry of equal-weight commodities portfolio", "Spot return of equal-weight commodities portfolio", "Carry of equal-weight commodities portfolio", "Excess return of long/short commodities portfolio", "Excess spot return of long/short commodities portfolio", "Interest rate adjusted carry of long/short commodities portfolio", "Aggregate backwardation/contango", "State of backwardation/contango", "State of inflation"],
    "columns": {
        "DATE": "date",
        "Excess return of equal-weight commodities portfolio": "excess_return_eqwt",
        "Excess spot return of equal-weight commodities portfolio": "excess_spot_return_eqwt",
        "Interest rate adjusted carry of equal-weight commodities portfolio": "ir_adjusted_carry_eqwt",
        "Spot return of equal-weight commodities portfolio": "spot_return_eqwt",
        "Carry of equal-weight commodities portfolio": "carry_eqwt",
        "Excess return of long/short commodities portfolio": "excess_return_long_short",
        "Excess spot return of long/short commodities portfolio": "excess_spot_return_long_short",
        "Interest rate adjusted carry of long/short commodities portfolio": "ir_adjusted_carry_long_short",
        "Aggregate backwardation/contango": "aggregate_backwardation_contango",
        "State of backwardation/contango": "state_backwardation_contango",
        "State of inflation": "state_inflation"
    }
}

def process_com(config):
    file_path = config['path']
    sheet_name = config['sheet']
    logger.info(f"Processing {file_path} (sheet {sheet_name})")
    try:
        df = pd.read_excel(file_path, sheet_name=sheet_name, header=None, skiprows=config['start_row'])
        df.columns = config['headers']
        logger.info(f"Read {len(df)} rows with set columns: {list(df.columns)}")
        
        expected_cols = list(config['columns'].keys())
        df = df[expected_cols]
        df.columns = [config['columns'][col] for col in df.columns]
        df['date'] = pd.to_datetime(df['date'], errors='coerce')
        df['associated_paper'] = config['paper']
        df = df.dropna(subset=['date'])
        logger.debug(f"After dropping NA dates, {len(df)} rows remain")
        
        with engine.connect() as connection:
            existing = pd.read_sql("SELECT date FROM aqr_cmdty_factors", connection)
            existing['date'] = pd.to_datetime(existing['date']).dt.strftime('%Y-%m-%d')
            existing_keys = set(existing['date'])
            logger.debug(f"Existing commodity dates sample (first 5): {list(existing_keys)[:5]}")
        
        df['date_str'] = df['date'].dt.strftime('%Y-%m-%d')
        df_new = df[~df['date_str'].isin(existing_keys)].drop(columns=['date_str'])
        
        if not df_new.empty:
            for chunk in [df_new[i:i+10000] for i in range(0, len(df_new), 10000)]:
                chunk.to_sql(
                    'aqr_cmdty_factors',
                    engine,
                    if_exists='append',
                    index=False,
                    dtype={
                        'date': DATE,
                        'excess_return_eqwt': DECIMAL(15, 6),
                        'excess_spot_return_eqwt': DECIMAL(15, 6),
                        'ir_adjusted_carry_eqwt': DECIMAL(15, 6),
                        'spot_return_eqwt': DECIMAL(15, 6),
                        'carry_eqwt': DECIMAL(15, 6),
                        'excess_return_long_short': DECIMAL(15, 6),
                        'excess_spot_return_long_short': DECIMAL(15, 6),
                        'ir_adjusted_carry_long_short': DECIMAL(15, 6),
                        'aggregate_backwardation_contango': DECIMAL(15, 6),
                        'state_backwardation_contango': VARCHAR(50),
                        'state_inflation': VARCHAR(50),
                        'associated_paper': VARCHAR(100)
                    }
                )
            logger.info(f"Loaded {len(df_new)} new rows into aqr_cmdty_factors from {file_path} (sheet {sheet_name})")
        else:
            logger.info(f"No new rows to load into aqr_cmdty_factors from {file_path} (sheet {sheet_name})")
            logger.debug(f"Input date range: {df['date'].min()} to {df['date'].max()}")
            logger.debug(f"Database date range: {existing['date'].min() if not existing.empty else 'N/A'} to {existing['date'].max() if not existing.empty else 'N/A'}")
            
    except Exception as e:
        logger.error(f"Error processing {file_path} (sheet {sheet_name}): {type(e).__name__}: {str(e)}")

# Run the script
process_com(config)
logger.info("COM data processing complete!")

In [9]:
# COM Version 2.0 (Draft)

import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.types import VARCHAR, DATE, DECIMAL
import logging
from dateutil.parser import parse
import re

logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler(), logging.FileHandler('load_com_data.log')]
)
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

engine = create_engine(
    "mssql+pyodbc://JULIANS_LAPTOP\\SQLEXPRESS/CWA_Fund_Database"
    "?driver=ODBC+Driver+18+for+SQL+Server&trusted_connection=yes&TrustServerCertificate=yes"
)

config = {
    "path": r"C:\Users\JulianHeron\Downloads\Commodities for the Long Run Index Level Data Monthly.xlsx",
    "sheet": 0,
    "start_row": 11,
    "paper": "Commodities for the Long Run",
    "headers": ["date", "Excess return of equal-weight commodities portfolio", "Excess spot return of equal-weight commodities portfolio", 
                "Interest rate adjusted carry of equal-weight commodities portfolio", "Spot return of equal-weight commodities portfolio", 
                "Carry of equal-weight commodities portfolio", "Excess return of long/short commodities portfolio", 
                "Excess spot return of long/short commodities portfolio", "Interest rate adjusted carry of long/short commodities portfolio", 
                "Aggregate backwardation/contango", "State of backwardation/contango", "State of inflation"],
    "columns": {
        "date": "date",
        "Excess return of equal-weight commodities portfolio": "excess_return_eqwt",
        "Excess spot return of equal-weight commodities portfolio": "excess_spot_return_eqwt",
        "Interest rate adjusted carry of equal-weight commodities portfolio": "ir_adjusted_carry_eqwt",
        "Spot return of equal-weight commodities portfolio": "spot_return_eqwt",
        "Carry of equal-weight commodities portfolio": "carry_eqwt",
        "Excess return of long/short commodities portfolio": "excess_return_long_short",
        "Excess spot return of long/short commodities portfolio": "excess_spot_return_long_short",
        "Interest rate adjusted carry of long/short commodities portfolio": "ir_adjusted_carry_long_short",
        "Aggregate backwardation/contango": "aggregate_backwardation_contango",
        "State of backwardation/contango": "state_backwardation_contango",
        "State of inflation": "state_inflation"
    }
}

def clean_date(date_val):
    if pd.isna(date_val) or str(date_val).strip().lower() in ['', 'nan']:
        return None
    try:
        return pd.to_datetime(date_val, errors='coerce')
    except:
        pass
    try:
        return parse(str(date_val), fuzzy=False)
    except:
        pass
    date_str = str(date_val).strip()
    if re.match(r'\d{4}-\d{2}-\d{2}', date_str):
        try:
            year, month, day = map(int, date_str.split('-'))
            if month > 12:
                month = 12
            if day > 31:
                day = 31
            return pd.to_datetime(f"{year}-{month:02d}-{day:02d}")
        except:
            pass
    logger.warning(f"Unable to parse date: {date_val}")
    return None

def process_com(config):
    file_path = config['path']
    sheet_name = config['sheet']
    logger.info(f"Processing {file_path} (sheet {sheet_name})")
    
    try:
        df = pd.read_excel(
            file_path,
            sheet_name=sheet_name,
            header=None,
            skiprows=config['start_row']
        )
        df.columns = config['headers']
        logger.info(f"Read {len(df)} rows with columns: {list(df.columns)}")

        # Generate dates for blank column A (1871-01-31 to 2018-12-31, monthly)
        df['date'] = pd.date_range(start='1871-01-31', periods=len(df), freq='M')
        
        expected_cols = list(config['columns'].keys())
        df = df[expected_cols]
        df.columns = [config['columns'][col] for col in df.columns]
        df['date'] = df['date'].apply(clean_date)
        df['associated_paper'] = config['paper']
        df = df.dropna(subset=['date'])
        logger.debug(f"After dropping NA dates, {len(df)} rows remain")
        
        # Debug: Log row counts per column
        for col in df.columns:
            logger.debug(f"Row count for {col}: {df[col].notna().sum()}")

        with engine.connect() as connection:
            with connection.begin() as transaction:
                existing = pd.read_sql("SELECT date FROM aqr_cmdty_factors", connection)
                existing['date'] = pd.to_datetime(existing['date'], errors='coerce').dt.date
                existing_keys = set(existing['date'])
                logger.debug(f"Existing commodity dates sample (first 5): {list(existing_keys)[:5]}")

                df_new = df[~df['date'].dt.date.isin(existing_keys)]
                
                if not df_new.empty:
                    logger.debug(f"New rows to insert: {len(df_new)}. Sample (first 5): {df_new.head().to_dict('records')}")
                    df_new.to_sql(
                        'aqr_cmdty_factors',
                        connection,
                        if_exists='append',
                        index=False,
                        dtype={
                            'date': DATE,
                            'excess_return_eqwt': DECIMAL(15, 6),
                            'excess_spot_return_eqwt': DECIMAL(15, 6),
                            'ir_adjusted_carry_eqwt': DECIMAL(15, 6),
                            'spot_return_eqwt': DECIMAL(15, 6),
                            'carry_eqwt': DECIMAL(15, 6),
                            'excess_return_long_short': DECIMAL(15, 6),
                            'excess_spot_return_long_short': DECIMAL(15, 6),
                            'ir_adjusted_carry_long_short': DECIMAL(15, 6),
                            'aggregate_backwardation_contango': DECIMAL(15, 6),
                            'state_backwardation_contango': VARCHAR(50),
                            'state_inflation': VARCHAR(50),
                            'associated_paper': VARCHAR(100)
                        }
                    )
                    logger.info(f"Inserted {len(df_new)} new rows into aqr_cmdty_factors from {file_path} (sheet {sheet_name})")
                else:
                    logger.info(f"No new rows to load into aqr_cmdty_factors from {file_path} (sheet {sheet_name})")
                    logger.debug(f"Input date range: {df['date'].min()} to {df['date'].max()}")
                    logger.debug(f"Database date range: {existing['date'].min() if not existing.empty else 'N/A'} to "
                                f"{existing['date'].max() if not existing.empty else 'N/A'}")
                
    except Exception as e:
        logger.error(f"Error processing {file_path} (sheet {sheet_name}): {type(e).__name__}: {str(e)}")

process_com(config)
logger.info("COM data processing complete!")

2025-04-11 18:43:00,225 - INFO - Processing C:\Users\JulianHeron\Downloads\Commodities for the Long Run Index Level Data Monthly.xlsx (sheet 0)
2025-04-11 18:43:00,534 - INFO - Read 1776 rows with columns: ['date', 'Excess return of equal-weight commodities portfolio', 'Excess spot return of equal-weight commodities portfolio', 'Interest rate adjusted carry of equal-weight commodities portfolio', 'Spot return of equal-weight commodities portfolio', 'Carry of equal-weight commodities portfolio', 'Excess return of long/short commodities portfolio', 'Excess spot return of long/short commodities portfolio', 'Interest rate adjusted carry of long/short commodities portfolio', 'Aggregate backwardation/contango', 'State of backwardation/contango', 'State of inflation']
C:\Users\JulianHeron\AppData\Local\Temp\ipykernel_40512\325418905.py:90: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df['date'] = pd.date_range(start='1871-01-31', periods